In [29]:
import pandas as pd

df_orders = pd.read_csv("orders.csv")
df_orders

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian
...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian


In [30]:
import psycopg2
import os
from dotenv import load_dotenv

load_dotenv()  # loads PGUSER, PGPASSWORD, PGDATABASE

conn = psycopg2.connect(
    dbname=os.getenv("PGDATABASE"),
    user=os.getenv("PGUSER"),
    password=os.getenv("PGPASSWORD"),
    host="localhost",
    port="5432"
)

import pandas as pd
query = "SELECT * FROM restaurants;"

df_restaurents = pd.read_sql(query, conn)

print(df_restaurents.head())

conn.close()

df_restaurents


   restaurant_id restaurant_name  cuisine  rating
0              1    Restaurant_1  Chinese     4.8
1              2    Restaurant_2   Indian     4.1
2              3    Restaurant_3  Mexican     4.3
3              4    Restaurant_4  Chinese     4.1
4              5    Restaurant_5  Chinese     4.8


C:\Users\kjupp\AppData\Local\Temp\ipykernel_18680\2375130865.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_restaurents = pd.read_sql(query, conn)


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8
...,...,...,...,...
495,496,Restaurant_496,Indian,3.1
496,497,Restaurant_497,Mexican,4.4
497,498,Restaurant_498,Chinese,3.9
498,499,Restaurant_499,Mexican,4.9


In [31]:
import pandas as pd

df_users = pd.read_json("users.json")

df_users

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold
...,...,...,...,...
2995,2996,User_2996,Hyderabad,Gold
2996,2997,User_2997,Hyderabad,Regular
2997,2998,User_2998,Bangalore,Regular
2998,2999,User_2999,Pune,Regular


In [32]:
df_merged = pd.merge(df_orders, df_users,on="user_id", how="left")
df_merged

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular
...,...,...,...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian,User_2528,Hyderabad,Gold
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi,User_2867,Bangalore,Regular
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese,User_522,Bangalore,Gold
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian,User_319,Bangalore,Gold


In [33]:
df = pd.merge(df_merged, df_restaurents, on="restaurant_id", how="left")
df

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian,User_2528,Hyderabad,Gold,Restaurant_249,Italian,4.7
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi,User_2867,Bangalore,Regular,Restaurant_267,Chinese,4.2
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese,User_522,Bangalore,Gold,Restaurant_420,Italian,4.0
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian,User_319,Bangalore,Gold,Restaurant_492,Italian,4.0


In [34]:
df.to_csv("final_food_delivery_dataset.csv")

In [36]:
#cleaning the dataset
df["order_date"] = pd.to_datetime(df["order_date"], format="%d-%m-%Y")

df


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,2023-02-18,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,2023-01-18,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,2023-07-15,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,2023-10-04,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,2023-12-25,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,2528,249,2023-05-21,1211.96,Royal Kitchen North Indian,User_2528,Hyderabad,Gold,Restaurant_249,Italian,4.7
9996,9997,2867,267,2023-08-06,1188.05,Darbar Cafe Punjabi,User_2867,Bangalore,Regular,Restaurant_267,Chinese,4.2
9997,9998,522,420,2023-11-11,979.44,Ruchi Tiffins Chinese,User_522,Bangalore,Gold,Restaurant_420,Italian,4.0
9998,9999,319,492,2023-09-08,1105.93,Swagath Kitchen North Indian,User_319,Bangalore,Gold,Restaurant_492,Italian,4.0


In [38]:
orders_over_time = (
    df
    .groupby(df["order_date"].dt.to_period("M"))
    .size()
    .reset_index(name="order_count")
)
orders_over_time

,order_date,order_count
0,2023-01,804
1,2023-02,785
2,2023-03,903
3,2023-04,812
4,2023-05,844
5,2023-06,784
6,2023-07,859
7,2023-08,851
8,2023-09,812
9,2023-10,863


In [57]:
revenue_trend = (
    df
    .groupby(df["order_date"].dt.to_period("M"))["total_amount"]
    .sum()
    .reset_index()
)

revenue_trend

df["quarter"] = df["order_date"].dt.quarter
quarter_revenue = df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False)
quarter_revenue
top_quarter = quarter_revenue.idxmax()
top_revenue = quarter_revenue.max()

print(f"Quarter with highest revenue: Q{top_quarter}")
print(f"Total revenue: ₹{top_revenue:.2f}")



Quarter with highest revenue: Q3
Total revenue: ₹2037385.10


In [ ]:
#Which city has the highest total revenue (total_amount) from Gold members?  
gold_city_revenue = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

gold_city_revenue


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [42]:
#Which cuisine has the highest average order value across all orders?
cuisine_aov = (
    df
    .groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

cuisine_aov


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [43]:
#How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?
user_total_spend = (
    df
    .groupby("user_id")["total_amount"]
    .sum()
)


high_value_users = user_total_spend[user_total_spend > 1000]


count_users = high_value_users.count()

count_users

2544

In [45]:
#Which restaurant rating range generated the highest total revenue?

rating_bins = [3.0, 3.5, 4.0, 4.5, 5.0]
rating_labels = [
    "3.0–3.5",
    "3.6–4.0",
    "4.1–4.5",
    "4.6–5.0"
]

df["rating_range"] = pd.cut(
    df["rating"],
    bins=rating_bins,
    labels=rating_labels,
    right=True,
    include_lowest=True
)
rating_revenue = (
    df
    .groupby("rating_range")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

rating_revenue


C:\Users\kjupp\AppData\Local\Temp\ipykernel_18680\3804809892.py:20: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("rating_range")["total_amount"]


rating_range
4.6–5.0    2197030.75
3.0–3.5    2136772.70
4.1–4.5    1960326.26
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

In [46]:
#Among Gold members, which city has the highest average order value?
gold_city_aov = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

gold_city_aov

city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [47]:
#Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?

restaurants_per_cuisine = (
    df.groupby("cuisine")["restaurant_id"]
      .nunique()
      .reset_index(name="restaurant_count")
)

revenue_per_cuisine = (
    df.groupby("cuisine")["total_amount"]
      .sum()
      .reset_index(name="total_revenue")
)

cuisine_analysis = restaurants_per_cuisine.merge(
    revenue_per_cuisine,
    on="cuisine"
)
cuisine_analysis.sort_values(
    by=["restaurant_count", "total_revenue"],
    ascending=[True, False]
)

cuisine_analysis


,cuisine,restaurant_count,total_revenue
0,Chinese,120,1930504.65
1,Indian,126,1971412.58
2,Italian,126,2024203.80
3,Mexican,128,2085503.09


In [50]:
#What percentage of total orders were placed by Gold members? (Rounded to nearest integer)
total_orders = df["order_id"].nunique()
gold_orders = df[df["membership"] == "Gold"]["order_id"].nunique()
percentage_gold_orders = round((gold_orders / total_orders) * 100)
percentage_gold_orders



50

In [54]:
options = [
    "Grand Cafe Punjabi",
    "Grand Restaurant South Indian",
    "Ruchi Mess Multicuisine",
    "Ruchi Foods Chinese"
]
df_options = df[df["restaurant_name_x"].isin(options)]
stats = (
    df_options
    .groupby("restaurant_name_x")
    .agg(
        avg_order_value=("total_amount", "mean"),
        total_orders=("order_id", "count")
    )
    .reset_index()
)
stats_filtered = stats[stats["total_orders"] < 20]

answer = stats_filtered.sort_values(
    by="avg_order_value",
    ascending=False
).head(1)

answer


,restaurant_name_x,avg_order_value,total_orders
2,Ruchi Foods Chinese,686.603158,19


In [55]:
top_combo = (
    df[
        ((df["membership"] == "Gold") & (df["cuisine"].isin(["Indian", "Italian"]))) |
        ((df["membership"] == "Regular") & (df["cuisine"].isin(["Indian", "Chinese"])))
    ]
    .groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .idxmax()
)

top_revenue = (
    df[
        ((df["membership"] == "Gold") & (df["cuisine"].isin(["Indian", "Italian"]))) |
        ((df["membership"] == "Regular") & (df["cuisine"].isin(["Indian", "Chinese"])))
    ]
    .groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .max()
)

print(f"Highest revenue combination: {top_combo[0]} + {top_combo[1]}")
print(f"Revenue: ₹{top_revenue:.2f}")

Highest revenue combination: Gold + Italian
Revenue: ₹1005779.05


In [58]:
#How many total orders were placed by users with Gold membership?
# Filter Gold members and count orders
gold_orders_count = df[df["membership"] == "Gold"]["order_id"].count()

print(f"Total orders placed by Gold members: {gold_orders_count}")


Total orders placed by Gold members: 4987


In [ ]:
hyderabad_revenue = round(df[df["city"] == "Hyderabad"]["total_amount"].sum())

hyderabad_revenue

Total revenue from Hyderabad: ₹1889367


In [60]:
distinct_users = df["user_id"].nunique()
distinct_users

2883

In [61]:
gold_aov = round(df[df["membership"] == "Gold"]["total_amount"].mean(), 2)
gold_aov

797.15

In [62]:
high_rating_orders = df[df["rating"] >= 4.5]["order_id"].count()
high_rating_orders

3374

In [ ]:

gold_df = df[df["membership"] == "Gold"]


top_city = gold_df.groupby("city")["total_amount"].sum().idxmax()


orders_in_top_city = gold_df[gold_df["city"] == top_city]["order_id"].count()

top_city


Number of orders in top revenue city (Chennai) among Gold members: 1337


In [ ]:
total_rows = df.shape[0]
total_rows


10000
